In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df=pd.read_csv("square_feet.csv",index_col='Users')
items=pd.read_csv("warehouses.csv")

In [3]:
df

,Dark_Store,Small_warehouse,Medium_warehouse,Large_warehouse,Distribution_centre_warehouse
Users,,,,,
70-262-7631,18699.0,NaN,NaN,95878.0,802714.0
67-529-8002,NaN,NaN,55569.0,NaN,NaN
71-032-7622,8081.0,43519.0,NaN,87702.0,NaN
57-852-3909,10944.0,35749.0,63975.0,NaN,NaN
52-706-8422,14566.0,37967.0,NaN,80937.0,878374.0
...,...,...,...,...,...
19-969-8322,18732.0,NaN,NaN,NaN,617132.0
97-991-2766,17505.0,28555.0,NaN,NaN,NaN
36-333-1725,NaN,NaN,NaN,NaN,NaN


In [4]:
df=df.fillna(0)

In [5]:
df['Dark_Store'] = df['Dark_Store'].astype('int')
df['Small_warehouse'] = df['Small_warehouse'].astype('int')
df['Medium_warehouse'] = df['Medium_warehouse'].astype('int')
df['Large_warehouse']=df['Large_warehouse'].astype('int')
df['Distribution_centre_warehouse']=df['Distribution_centre_warehouse'].astype('int')


In [6]:
total = df['Dark_Store'] + df['Small_warehouse']+df['Medium_warehouse']+df['Large_warehouse']+df['Distribution_centre_warehouse']
df = df.pipe(lambda x: (x*100).div(total, axis='index'))


In [7]:
df

,Dark_Store,Small_warehouse,Medium_warehouse,Large_warehouse,Distribution_centre_warehouse
Users,,,,,
70-262-7631,2.038503,0.000000,0.000000,10.452299,87.509198
67-529-8002,0.000000,0.000000,100.000000,0.000000,0.000000
71-032-7622,5.801065,31.240757,0.000000,62.958177,0.000000
57-852-3909,9.889037,32.302924,57.808038,0.000000,0.000000
52-706-8422,1.439550,3.752258,0.000000,7.998960,86.809231
...,...,...,...,...,...
19-969-8322,2.945913,0.000000,0.000000,0.000000,97.054087
97-991-2766,38.004776,61.995224,0.000000,0.000000,0.000000
36-333-1725,NaN,NaN,NaN,NaN,NaN


In [8]:
df=df.fillna(0)

In [9]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_std = df.apply(standardize).T
print(df_std)

sparse_df = sparse.csr_matrix(df_std.values)
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df),index=df.columns,columns=df.columns)
corrMatrix

Users                          70-262-7631  67-529-8002  71-032-7622  \
Dark_Store                       -0.022069    -0.042454     0.015556   
Small_warehouse                  -0.120431    -0.120431     0.191976   
Medium_warehouse                 -0.206394     0.793606    -0.206394   
Large_warehouse                  -0.081998    -0.186521     0.443061   
Distribution_centre_warehouse     0.438893    -0.436199    -0.436199   

Users                          57-852-3909  52-706-8422  44-801-7398  \
Dark_Store                        0.056436    -0.028059    -0.039124   
Small_warehouse                   0.202598    -0.082909    -0.080530   
Medium_warehouse                  0.371686    -0.206394    -0.206394   
Large_warehouse                  -0.186521    -0.106532    -0.186521   
Distribution_centre_warehouse    -0.436199     0.431893     0.520569   

Users                          32-006-4001  51-912-0166  31-892-4916  \
Dark_Store                       -0.023399    -0.042454     0.

,Dark_Store,Small_warehouse,Medium_warehouse,Large_warehouse,Distribution_centre_warehouse
Dark_Store,1.000000,0.075644,-0.002146,-0.061744,-0.291059
Small_warehouse,0.075644,1.000000,0.000284,-0.039855,-0.462501
Medium_warehouse,-0.002146,0.000284,1.000000,-0.071725,-0.571299
Large_warehouse,-0.061744,-0.039855,-0.071725,1.000000,-0.510559
Distribution_centre_warehouse,-0.291059,-0.462501,-0.571299,-0.510559,1.000000


In [11]:
def get_similar(movie_name,rating):
    similar_score = corrMatrix[movie_name]*(rating-50.5) #50..5 is the avg of percentage (0-100)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score
get_similar("Dark_Store",78)

Dark_Store                       27.500000
Small_warehouse                   2.080218
Medium_warehouse                 -0.059007
Large_warehouse                  -1.697956
Distribution_centre_warehouse    -8.004123
Name: Dark_Store, dtype: float64

In [12]:
action_lover = [("Dark_Store",2.038503),("Small_warehouse",0),("Medium_warehouse",0),("Large_warehouse",10.452299)]
similar_scores = pd.DataFrame()
for movie,rating in action_lover:
    similar_scores = similar_scores.append(get_similar(movie,rating),ignore_index = True)

similar_scores.head(10)

/var/folders/yg/rsw8j92510j7cjvvkgjr4s2r0000gn/T/ipykernel_4425/3403360689.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_scores = similar_scores.append(get_similar(movie,rating),ignore_index = True)


,Distribution_centre_warehouse,Large_warehouse,Medium_warehouse,Small_warehouse,Dark_Store
0,14.105155,2.992200,0.103984,-3.665836,-48.461497
1,23.356297,2.012695,-0.014347,-50.500000,-3.820037
2,28.850593,3.622137,-50.500000,-0.014347,0.108358
3,20.446694,-40.047701,2.872441,1.596115,2.472699


In [13]:
similar_scores.sum().sort_values(ascending=False)[:2]

Distribution_centre_warehouse    86.758740
Large_warehouse                 -31.420669
dtype: float64